<a href="https://colab.research.google.com/github/mahomudgamalfcis/Image-Paragraph-Captioning/blob/master/DemoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1- Change Version of CUDA from 10.0 to 9.2


In [0]:
# Check for Current version
!nvcc --version
#--------------------------------------------------

# Uninstall the current version of CUDA
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update
#---------------------------------------------------

# Install the new version of CUDA
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9-2
#---------------------------------------------------

# check if CUDA 9.2 is installed
!nvcc --version


#2- Setup Torch and It's Packages


In [0]:
# This Commands will install Torch with luaJIT
!git clone https://github.com/nagadomi/distro.git ~/torch --recursive
%cd ~/torch
!bash install-deps
!./install.sh
#--------------------------------------------------

In [0]:
#this will install torch-rnn packcage
!/root/torch/install/bin/luarocks  install https://raw.githubusercontent.com/jcjohnson/torch-rnn/master/torch-rnn-scm-1.rockspec

In [0]:
''' this command is to install stnbhw-scm-1, but it is a modified repo. we modified the CMakeLists file to change GPU architecture to 
fit the one which installed in Googel Colab. the line we changed is 
 IF (CUDA_FOUND)
   LIST(APPEND CUDA_NVCC_FLAGS "-arch=sm_20") To 

 IF (CUDA_FOUND)
   LIST(APPEND CUDA_NVCC_FLAGS "-arch=sm_30")'''
!/root/torch/install/bin/luarocks install https://raw.githubusercontent.com/mahomudgamalfcis/stnbhwd/master/stnbhwd-scm-1.rockspec

In [0]:
# the commands below will install HDF5-1.10 
%cd /content
!git clone https://github.com/anibali/torch-hdf5.git
%cd torch-hdf5
!git checkout hdf5-1.10 
!/root/torch/install/bin/luarocks make hdf5-0-0.rockspec


In [0]:
# this command to display Installed packages by Luarocks
!/root/torch/install/bin/luarocks list

#3- Clone im2p repository 

In [0]:
%cd /content
!git clone https://github.com/mahomudgamalfcis/im2p.git

### i- Download pre-trained model of Encoder

In [0]:
%cd /content/im2p
!mkdir -p data/models/densecap
%cd ./data/models/densecap
!wget http://cs.stanford.edu/people/jcjohns/densecap/densecap-pretrained-vgg16.t7.zip
!unzip densecap-pretrained-vgg16.t7.zip
!rm densecap-pretrained-vgg16.t7.zip
%cd ../../../

###ii-Attach Dataset from Drive

In [0]:
# Mount The place where our dataset exist..
from google.colab import drive
drive.mount('/content/drive')

###iii- Extract feature for Training dataset

In [0]:
# Read train_split file which contains all ids of training images
train_imgs_ID = "/content/im2p/data/train_split.json"
with open(train_imgs_ID,'r') as all_train_imgs:
    trainImageID = json.load(all_train_imgs)
print(len(trainImageID))
#-----------------------------------------

# pick all training Images from all images
train_images_path = "/content/drive/My Drive/VG/Images"
train_images = glob.glob(train_images_path + "/*.jpg")
f = open("imgs_train_path.txt", "w")
for img in train_images:
  img_name = int(os.path.basename(img).split(".")[0])
  if img_name in trainImageID:
      f.write(img + "\n")
f.close()
# At this step we couldn't extrat the features of all training images in one step
# so we divide it into 8 files each file contain set of images. but the previous code 
# read all images and save it in one text file. 

%cd /content/im2p
!/root/torch/install/bin/th /content/im2p/extract_features.lua -boxes_per_image 50 -max_images -1 -checkpoint /content/im2p/data/models/densecap/densecap-pretrained-vgg16.t7  -input_txt /content/im2p/imgs_train_path.txt \
                          -output_h5 ./data/im2p_train_output.h5 -gpu 0 -use_cudnn 1

###vi- Extract feature for Testing dataset

In [0]:
# Read train_split file which contains all ids of training images
train_imgs_ID = "/content/im2p/data/test_split.json"
with open(train_imgs_ID,'r') as all_train_imgs:
    testImageID = json.load(all_train_imgs)
print(len(testImageID))
#-----------------------------------------

# pick all training Images from all images
train_images_path = "/content/drive/My Drive/VG/Images"
train_images = glob.glob(train_images_path + "/*.jpg")
f = open("imgs_train_path.txt", "w")
for img in train_images:
  img_name = int(os.path.basename(img).split(".")[0])
  if img_name in testImageID:
      f.write(img + "\n")
f.close()
%cd /content/im2p
!/root/torch/install/bin/th /content/im2p/extract_features.lua -boxes_per_image 50 -max_images -1 -checkpoint /content/im2p/data/models/densecap/densecap-pretrained-vgg16.t7  -input_txt /content/im2p/imgs_test_path.txt \
                          -output_h5 ./data/im2p_test_output.h5 -gpu 0 -use_cudnn 1

#4- Check the Output of Encoder

In [0]:

f = h5py.File(r'/content/drive/My Drive/VG/im2p_test_output.h5')
list(f)
f['feats'].shape